In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
'''for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))'''

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

"for dirname, _, filenames in os.walk('/kaggle/input'):\n    for filename in filenames:\n        print(os.path.join(dirname, filename))"

In [4]:
filenames=os.listdir('/kaggle/input/agriculture-crop-images')
filenames

['test_crop_image',
 'some_more_images',
 'Crop_details.csv',
 'crop_images',
 'kag2']

In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [7]:
data_gen=ImageDataGenerator(zoom_range=0.2,
                            rotation_range=0.2,
                            rescale=1/255)

In [8]:
gen1=data_gen.flow_from_directory('/kaggle/input/agriculture-crop-images/crop_images',
                                 target_size=(224,224),batch_size=64)

Found 201 images belonging to 5 classes.


In [9]:
gen2=data_gen.flow_from_directory('/kaggle/input/agriculture-crop-images/kag2',
                                 target_size=(224,224),batch_size=64)

Found 804 images belonging to 5 classes.


In [10]:
gen3=data_gen.flow_from_directory('/kaggle/input/agriculture-crop-images/some_more_images',
                                 target_size=(224,224),batch_size=64)

Found 50 images belonging to 1 classes.


In [11]:
gen1.class_indices

{'jute': 0, 'maize': 1, 'rice': 2, 'sugarcane': 3, 'wheat': 4}

In [12]:
gen2.class_indices

{'jute': 0, 'maize': 1, 'rice': 2, 'sugarcane': 3, 'wheat': 4}

In [13]:
gen3.class_indices

{'some_more_images': 0}

In [14]:
from tensorflow.keras.applications import ResNet50

In [15]:
resnet_model=ResNet50(include_top=False,weights='imagenet',
                      input_shape=(224,224,3))

94765736/94765736 [==============================] - 1s 0us/step


In [16]:
for layer in resnet_model.layers:
    layer.trainable=False

In [17]:
resnet_model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                           

In [18]:
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.models import Model

In [20]:
flatten_layer=Flatten()(resnet_model.output)

In [21]:
last_layer=Dense(5,activation='softmax')(flatten_layer)

In [22]:
model=Model(inputs=resnet_model.input,outputs=last_layer)
model.compile(loss='categorical_crossentropy',metrics=['accuracy'])

In [23]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                              

In [25]:
model.fit(gen1,epochs=30)

Epoch 1/30
4/4 [==============================] - 3s 897ms/step - loss: 9.7714 - accuracy: 0.1791
Epoch 2/30
4/4 [==============================] - 3s 610ms/step - loss: 9.8699 - accuracy: 0.2189
Epoch 3/30
4/4 [==============================] - 3s 892ms/step - loss: 12.9312 - accuracy: 0.2040
Epoch 4/30
4/4 [==============================] - 3s 596ms/step - loss: 6.4783 - accuracy: 0.2388
Epoch 5/30
4/4 [==============================] - 3s 613ms/step - loss: 9.2011 - accuracy: 0.1940
Epoch 6/30
4/4 [==============================] - 3s 594ms/step - loss: 8.2684 - accuracy: 0.2388
Epoch 7/30
4/4 [==============================] - 3s 610ms/step - loss: 7.7308 - accuracy: 0.2289
Epoch 8/30
4/4 [==============================] - 3s 589ms/step - loss: 8.1424 - accuracy: 0.2289
Epoch 9/30
4/4 [==============================] - 3s 637ms/step - loss: 7.5436 - accuracy: 0.2438
Epoch 10/30
4/4 [==============================] - 3s 557ms/step - loss: 8.2389 - accuracy: 0.2537
Epoch 11/30
4/4 [=

In [26]:
model.fit(gen2,epochs=30)

Epoch 1/30
13/13 [==============================] - 15s 1s/step - loss: 5.8547 - accuracy: 0.3308
Epoch 2/30
13/13 [==============================] - 11s 797ms/step - loss: 7.8890 - accuracy: 0.2935
Epoch 3/30
13/13 [==============================] - 12s 875ms/step - loss: 6.8577 - accuracy: 0.3010
Epoch 4/30
13/13 [==============================] - 11s 826ms/step - loss: 8.4440 - accuracy: 0.2612
Epoch 5/30
13/13 [==============================] - 11s 870ms/step - loss: 7.7958 - accuracy: 0.2910
Epoch 6/30
13/13 [==============================] - 11s 824ms/step - loss: 6.7531 - accuracy: 0.3122
Epoch 7/30
13/13 [==============================] - 11s 834ms/step - loss: 7.7606 - accuracy: 0.2575
Epoch 8/30
13/13 [==============================] - 12s 884ms/step - loss: 7.0030 - accuracy: 0.2848
Epoch 9/30
13/13 [==============================] - 11s 828ms/step - loss: 7.4793 - accuracy: 0.3097
Epoch 10/30
13/13 [==============================] - 12s 844ms/step - loss: 6.0435 - accuracy:

In [27]:
model.save('TL_crop_resnet50.h5')

In [28]:
test_files=list(os.listdir('/kaggle/input/agriculture-crop-images/test_crop_image'))
test_files

['maize000.jfif',
 'rice-5174887_1280.jpg',
 'juteplants.jpg',
 'maize images.jfif',
 'sugarcaneplant.jpg',
 'maize corn set2.jpg',
 'wheat-field-artificial-irrigation-rural-electrification-to-harvest-166395991.jpg',
 'juteimg.jpg',
 'wheatcropfield04.jpg',
 'rice-4967164_1280.jpg',
 'rice-fields-204128_1280.jpg',
 'jute002.jpg',
 'wheatarial02.jpg',
 'juteplant.jpg',
 'sugarcane fields.jpg',
 'sugarcaneplants.jpg',
 'sugarcanefield02.jpg',
 'maize plant set.jpg',
 'maizecornleaves.jfif',
 'wheat-field01.jpg',
 'wheatcropfield.jpg',
 'jute-field.jpg',
 'rice-field01.jpg',
 'wheatcrop01.jpg',
 'jutefield.jpg',
 'sugarcane-field8.jpg',
 'jute003.jpg',
 'sugarcane-farm-in-the-mountain-countryside-of-thailand.jpg',
 'wheat-field02.jpg',
 'jute03.jpg',
 'wheat-field-395545_960_720.jpg',
 'juteleaves.jpg',
 'maize00corn-fields.jpg',
 'maize-field01.jpg',
 'rice-field02.jpg',
 'wheat.jpg',
 'rice8122f869e3f.jpg',
 'wheatcrops.jpg',
 'wheatss.jpg',
 'sugarcane-field.jpg',
 'maize-field.jpg',
 

In [29]:
from tensorflow.keras.models import load_model
import cv2

In [30]:
model=load_model('TL_crop_resnet50.h5')

In [33]:
def get_predict(A):
    A=cv2.imread('/kaggle/input/agriculture-crop-images/test_crop_image/'+file)
    A=cv2.resize(A,(224,224))
    A=A.reshape(1,224,224,3)
    yp=model.predict(A).argmax()
    
    if yp==0:
        return 'jute'
    elif yp==1:
        return 'maize'
    elif yp==2:
        return 'rice'
    elif yp==3:
        return 'sugercane'
    else:
        return 'wheat'
    
    

In [34]:
f_names=os.listdir('/kaggle/input/agriculture-crop-images/test_crop_image/')
result=[]
for file in f_names:
    result.append([file,get_predict(file)])
result

1/1 [==============================] - 0s 25ms/step


[['maize000.jfif', 'sugercane'],
 ['rice-5174887_1280.jpg', 'sugercane'],
 ['juteplants.jpg', 'sugercane'],
 ['maize images.jfif', 'sugercane'],
 ['sugarcaneplant.jpg', 'sugercane'],
 ['maize corn set2.jpg', 'sugercane'],
 ['wheat-field-artificial-irrigation-rural-electrification-to-harvest-166395991.jpg',
  'sugercane'],
 ['juteimg.jpg', 'sugercane'],
 ['wheatcropfield04.jpg', 'sugercane'],
 ['rice-4967164_1280.jpg', 'sugercane'],
 ['rice-fields-204128_1280.jpg', 'sugercane'],
 ['jute002.jpg', 'sugercane'],
 ['wheatarial02.jpg', 'sugercane'],
 ['juteplant.jpg', 'sugercane'],
 ['sugarcane fields.jpg', 'maize'],
 ['sugarcaneplants.jpg', 'sugercane'],
 ['sugarcanefield02.jpg', 'maize'],
 ['maize plant set.jpg', 'maize'],
 ['maizecornleaves.jfif', 'sugercane'],
 ['wheat-field01.jpg', 'sugercane'],
 ['wheatcropfield.jpg', 'sugercane'],
 ['jute-field.jpg', 'sugercane'],
 ['rice-field01.jpg', 'sugercane'],
 ['wheatcrop01.jpg', 'sugercane'],
 ['jutefield.jpg', 'sugercane'],
 ['sugarcane-field